# Defining a custom example model in DMGP
In this notebook, we are looking at how to define a custom model in DMGP. As an example, we consider a 2-layer sparse DGP model for a regression task. The model consists of two layers, each with a level-3 sparse grid design.

In [1]:
import torch
import torch.nn as nn
from dmgp.layers.linear import LinearFlipout
from dmgp.layers.activation import TMK

## Defining a 2-layer DMGP Model

In the next cell, we define our simple exact DMGP for regression. The model consists of two layers, each with level-3 sparse grid design. 

In [2]:
from dmgp.utils.sparse_design.design_class import HyperbolicCrossDesign
from dmgp.kernels.laplace_kernel import LaplaceProductKernel

# Define a 2-layer DMGP model for regression
class DMGP_regression(nn.Module):
    def __init__(self, input_dim, output_dim, design_class, kernel):
        super(DMGP_regression, self).__init__()
        
        # 1st layer of DGP: input:[n, input_dim] size tensor, output:[n, 8] size tensor
        self.tmk1 = TMK(in_features=input_dim, n_level=3, design_class=design_class, kernel=kernel)
        self.fc1 = LinearFlipout(self.tmk1.out_features, 7)

        # 2nd layer of DGP: input:[n, 8] size tensor, output:[n, output_dim] size tensor
        self.tmk2 = TMK(in_features=7, n_level=3, design_class=design_class, kernel=kernel)
        self.fc2 = LinearFlipout(in_features=self.tmk2.out_features, out_features=output_dim)

    def forward(self, x):
        kl_sum = 0
        x = self.tmk1(x)
        x, kl = self.fc1(x)
        kl_sum += kl
        x = self.tmk2(x)
        x, kl = self.fc2(x)
        kl_sum += kl
        return torch.squeeze(x), kl_sum
    
model = DMGP_regression(input_dim=1, 
                        output_dim=1, 
                        design_class=HyperbolicCrossDesign, 
                        kernel=LaplaceProductKernel(1.))

## Viewing model hyperparameters
Let's take a look at the model parameters. By "parameters", here I mean explicitly objects of type `torch.nn.Parameter` that will have gradients filled in by autograd. To access these, we use `model.state_dict()` which returns a dictionary of the model's parameters.

In [3]:
model_params = model.state_dict()

## Counting the number of parameters
We can count the total number of parameters in the model by counting the number of parameters in each layer.

In [4]:
def parameter_count(model):
    all_parameters = list(model.parameters())
    layer_parameters=[len(i) for i in all_parameters]
    print(layer_parameters)
    print("Total number of parameters in the network: ", sum(layer_parameters))
    return sum(layer_parameters)

parameter_count(model)

[7, 7, 7, 7, 1, 1, 1, 1]
Total number of parameters in the network:  32


32

## Viewing model architecture
We can also print the model architecture by simply printing the model object.

In [5]:
print(model)

DMGP_regression(
  (tmk1): TMK(
    (kernel): LaplaceProductKernel()
  )
  (fc1): LinearFlipout()
  (tmk2): TMK(
    (kernel): LaplaceProductKernel()
  )
  (fc2): LinearFlipout()
)


## Saving Model State
The state dictionary above represents all trainable parameters in the model. We can save this state dictionary to a file and load it back later.

In [6]:
torch.save(model.state_dict(), 'model_state.pth')

## Loading Model State
Next, we load this state in to a new model and demonstrate that the parameters were updated correctly.

In [7]:
state_dict = torch.load('model_state.pth')
model = DMGP_regression(input_dim=1, 
                        output_dim=1, 
                        design_class=HyperbolicCrossDesign, 
                        kernel=LaplaceProductKernel(1.))
model.load_state_dict(state_dict)

<All keys matched successfully>

In [8]:
model.state_dict()

OrderedDict([('tmk1.design_points',
              tensor([[ 0.0000],
                      [-1.0000],
                      [ 1.0000],
                      [-1.5000],
                      [-0.5000],
                      [ 0.5000],
                      [ 1.5000]])),
             ('tmk1.chol_inv',
              tensor(indices=tensor([[0, 0, 1, 0, 2, 1, 3, 1, 4, 0, 0, 5, 2, 2, 6, 0, 0, 0, 0],
                                     [0, 1, 1, 2, 2, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 0, 0, 0, 0]]),
                     values=tensor([ 1.0000, -0.3956,  1.0754, -0.3956,  1.0754, -0.7629,
                                     1.2578, -0.6523,  1.4710, -0.6523, -0.6523,  1.4710,
                                    -0.6523, -0.7629,  1.2578,  0.0000,  0.0000,  0.0000,
                                     0.0000]),
                     size=(7, 7), nnz=19, layout=torch.sparse_coo)),
             ('fc1.mu_weight',
              tensor([[-0.0758,  0.0461, -0.1482,  0.0683, -0.1386, -0.1030,  0.0920],
 

See our [documentation](https://sparse-dgp.readthedocs.io/en/latest/) for more information on how to use the library.